In [1]:
import sys
sys.path.append('/Users/user/meepc')
import numpy as np
import pandas as pd
from models import Hankel,Corrhankel,RobustPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def calculate_fpr(y_actual, y_prediction):
    fp = 0
    tn = 0
    
    for i in range(len(y_actual)):
        if y_actual[i] == 0 and y_prediction[i] == 1:
            fp += 1
        elif y_actual[i] == 0 and y_prediction[i] == 0:
            tn += 1
    fpr= fp/(fp+tn)
    return fpr

In [3]:
hankel = Hankel()
corrhankel = Corrhankel() 
lag = 360
stride = 0.5

In [4]:
train_normal = pd.read_csv('~/data/te/normal_training.csv')
train_attack = pd.read_csv('~/data/te/attack_training.csv')
train_normal = pd.concat((train_normal,train_attack),axis=0,ignore_index=True)
y_truth = train_normal.loc[:,'LABEL']

In [5]:
sensors = [col for col in train_normal.columns if col not in ['faultNumber', 'simulationRun', 'sample','LABEL']]

In [6]:
#attack indexes in original training data
temp = hankel.fit(np.array(y_truth) ,lag,stride)
attack_idx = np.any(temp>0,axis=0).astype(int)
attack_idx = np.where(attack_idx>0)[0]
attack_idx

array([1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397,
       1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408,
       1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419,
       1420, 1421, 1422, 1423, 1424, 1425, 1426, 1427, 1428, 1429, 1430,
       1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441,
       1442], dtype=int64)

In [7]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))

In [8]:
#  train_attack[sensors]
X_attack = pd.DataFrame(index=train_attack.index, columns = sensors, data=scaler.fit_transform(train_attack[sensors])).reset_index().drop(columns=['index'])

In [9]:
corr_normal,nolag_normal = corrhankel.fit(X_normal.to_numpy(),lag,stride)

In [10]:
corr_attack,nolag_attack = corrhankel.fit(X_attack.to_numpy(),lag,stride)

In [11]:
df_test1 = pd.read_csv('~/data/te/normal_testing.csv')
df_test2 = pd.read_csv('~/data/te/attack_testing.csv')
df_test=pd.concat((df_test1,df_test2),axis=0)

# Epasad with 1 cluster (Normal data)

In [12]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = df_test.loc[:,'LABEL']

In [13]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [14]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
fpr = []
cluster_indexes=[]
pca_counts=0
meepc_counts=0

for i,sens in enumerate(sensors):
    print("sensor -> ", sens)
    train_normal = X_normal.loc[:,sens].values
    model = RobustPipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,kscore_init='inertia',tune=False,alpha=0.08,corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,y_truth=y_truth)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    fpr.append(calculate_fpr(y_actual, y_predicted))

    #clustering
    cluster_indexes.append(model.cluster_idx)
    #pca

    pca_counts=max(pca_counts,model.total_pca_attacks)
    
    #meepc
    meepc_counts=max(meepc_counts,model.counts_meepc)
    
    

sensor ->  xmeas_1
------[CLUSTER] attack points found is 31 
------[PCA] attack points found is 56 
------[Meepc] attack points found is 37 
sensor ->  xmeas_2
------[CLUSTER] attack points found is 19 
------[PCA] attack points found is 56 
------[Meepc] attack points found is 11 
sensor ->  xmeas_3
------[CLUSTER] attack points found is 24 
------[PCA] attack points found is 56 
------[Meepc] attack points found is 39 
sensor ->  xmeas_4
------[CLUSTER] attack points found is 29 
------[PCA] attack points found is 56 
------[Meepc] attack points found is 37 
sensor ->  xmeas_5
------[CLUSTER] attack points found is 10 
------[PCA] attack points found is 56 
------[Meepc] attack points found is 7 
sensor ->  xmeas_6
------[CLUSTER] attack points found is 24 
------[PCA] attack points found is 56 
------[Meepc] attack points found is 15 
sensor ->  xmeas_7
------[CLUSTER] attack points found is 30 
------[PCA] attack points found is 56 
------[Meepc] attack points found is 36 
sensor 

In [15]:
attack_idx 

array([1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397,
       1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408,
       1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419,
       1420, 1421, 1422, 1423, 1424, 1425, 1426, 1427, 1428, 1429, 1430,
       1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441,
       1442], dtype=int64)

In [16]:
cluster_inactive_idx = np.unique(np.concatenate(cluster_indexes))
commonidxs=np.intersect1d(attack_idx,cluster_inactive_idx)
percentage = len(commonidxs)/len(attack_idx)*100
print(percentage)
print(commonidxs)



98.21428571428571
[1387 1388 1389 1390 1391 1392 1393 1394 1395 1396 1397 1398 1399 1400
 1401 1402 1403 1404 1405 1406 1407 1408 1409 1410 1411 1412 1413 1414
 1415 1416 1417 1418 1419 1420 1421 1422 1423 1424 1425 1426 1427 1428
 1429 1431 1432 1433 1434 1435 1436 1437 1438 1439 1440 1441 1442]


In [17]:
percentage = pca_counts/len(attack_idx)*100
print(percentage)



100.0


In [18]:
percentage = meepc_counts/len(attack_idx)*100
print(percentage) 

76.78571428571429


In [19]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0])

In [20]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))
print("False Positive Rate ",calculate_fpr(y_actual, y_predicted))

Accuracy  0.18181818181818182
Precision  1.0
Recall  0.1509433962264151
F1-score  0.26229508196721313
False Positive Rate  0.0


In [21]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.04073426573426574 Median : 0.03636363636363636 Min : 0.03636363636363636 Max : 0.09090909090909091
precision - Mean: 0.19230769230769232 Median : 0.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.0045355587808417995 Median : 0.0 Min : 0.0 Max : 0.05660377358490566
f1 -        Mean: 0.008778879486459012 Median : 0.0 Min : 0.0 Max : 0.10714285714285715


# Multiple clusters + No threshold tuning

In [22]:
cluster_accuracy = []
cluster_precision = []
cluster_recall = []
cluster_fscore = []
cluster_fpr = []
for k in range(2,5):
    print('-------------------------------------',k,'-------------------------------------')
    
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    cluster_indexes=[]
    pca_counts=0
    meepc_counts=0
    for i,sens in enumerate(sensors):
        print(sens)
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = RobustPipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',alpha=0.08,corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,y_truth=y_truth)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted)) 
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual, y_predicted))

        cluster_indexes.append(model.cluster_idx)
        pca_counts=max(pca_counts,model.total_pca_attacks)
        meepc_counts=max(meepc_counts,model.counts_meepc)
    

    #cluster
    cluster_inactive_idx = np.unique(np.concatenate(cluster_indexes))
    commonidxs=np.intersect1d(attack_idx,cluster_inactive_idx)
    percentage = len(commonidxs)/len(attack_idx)*100
    print("Cluster - attack points detected {} %".format(percentage))
    #PCA
    
    percentage = pca_counts/len(attack_idx)*100
    print("PCA - attack points detected {} %".format(percentage))
    #Meepc
    percentage = meepc_counts/len(attack_idx)*100
    print("Meepc - attack points detected {} %".format(percentage),"/n")
            
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy.append(accuracy_score(y_actual, y_predicted))
    cluster_precision.append(precision_score(y_actual,y_predicted))
    cluster_recall.append(recall_score(y_actual,y_predicted))
    cluster_fscore.append(f1_score(y_actual,y_predicted))
    cluster_fpr.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy[-1])
    print("Precision ",cluster_precision[-1])
    print("Recall ",cluster_recall[-1])
    print("F1-score ",cluster_fscore[-1])
    print("False Positive Rate ",cluster_fpr[-1],"/n") 
    

------------------------------------- 2 -------------------------------------
xmeas_1
------[CLUSTER] attack points found is 31 
------[PCA] attack points found is 22 
------[Meepc] attack points found is 12 
------[PCA] attack points found is 34 
------[Meepc] attack points found is 16 
xmeas_2
------[CLUSTER] attack points found is 19 
------[PCA] attack points found is 26 
------[Meepc] attack points found is 3 
------[PCA] attack points found is 30 
------[Meepc] attack points found is 2 
xmeas_3
------[CLUSTER] attack points found is 24 
------[PCA] attack points found is 24 
------[Meepc] attack points found is 11 
------[PCA] attack points found is 32 
------[Meepc] attack points found is 10 
xmeas_4
------[CLUSTER] attack points found is 30 
------[PCA] attack points found is 32 
------[Meepc] attack points found is 19 
------[PCA] attack points found is 24 
------[Meepc] attack points found is 16 
xmeas_5
------[CLUSTER] attack points found is 11 
------[PCA] attack points fou

In [23]:
print("Accuracy Scores: ",cluster_accuracy)
print("Precision Scores: ",cluster_precision)
print("Recall Scores: ",cluster_recall)
print("F1 Scores: ",cluster_fscore)
print("False Positive Rates: ",cluster_fpr)

Accuracy Scores:  [0.3090909090909091, 0.39090909090909093, 0.32727272727272727]
Precision Scores:  [1.0, 1.0, 1.0]
Recall Scores:  [0.2830188679245283, 0.36792452830188677, 0.3018867924528302]
F1 Scores:  [0.4411764705882353, 0.5379310344827586, 0.463768115942029]
False Positive Rates:  [0.0, 0.0, 0.0]
